In [ ]:
!pip install langchain langchain_core langchain_groq langchain_community langgraph gradio

In [ ]:
# Imports and setup
from typing import TypedDict, Dict
from langgraph.graph import StateGraph, END
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.graph import MermaidDrawMethod
from IPython.display import display, Image

In [ ]:
# Define shared state
class State(TypedDict):
    query: str
    category: str
    sentiment: str
    response: str

In [ ]:
# Initialize Groq LLM
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key="YOUR_GROQ_API_KEY",
    model_name="llama-3.3-70b-versatile"
)

# Test connection
llm.invoke("What is LangChain?").content

In [ ]:
# Node: Categorize query
def categorize(state: State) -> State:
    prompt = ChatPromptTemplate.from_template(
        "Categorize the following customer query into one of these categories: "
        "Technical, Billing, General. Query: {query}"
    )
    category = (prompt | llm).invoke({"query": state['query']}).content
    return {"category": category}

# Node: Analyze sentiment
def analyze_sentiment(state: State) -> State:
    prompt = ChatPromptTemplate.from_template(
        "Analyze the sentiment of the following customer query. "
        "Respond with Positive, Neutral, or Negative. Query: {query}"
    )
    sentiment = (prompt | llm).invoke({"query": state['query']}).content
    return {"sentiment": sentiment}

In [ ]:
# Support handlers
def handle_technical(state: State) -> State:
    prompt = ChatPromptTemplate.from_template(
        "Provide a technical support response to the following query: {query}"
    )
    response = (prompt | llm).invoke({"query": state['query']}).content
    return {"response": response}

def handle_billing(state: State) -> State:
    prompt = ChatPromptTemplate.from_template(
        "Provide a billing support response to the following query: {query}"
    )
    response = (prompt | llm).invoke({"query": state['query']}).content
    return {"response": response}

def handle_general(state: State) -> State:
    prompt = ChatPromptTemplate.from_template(
        "Provide a general support response to the following query: {query}"
    )
    response = (prompt | llm).invoke({"query": state['query']}).content
    return {"response": response}

def escalate(state: State) -> State:
    return {"response": "This query has been escalated to a human agent due to negative sentiment."}

In [ ]:
# Routing logic
def route_query(state: State):
    if state['sentiment'] == "Negative":
        return "escalate"
    elif state['category'] == "Technical":
        return "handle_technical"
    elif state['category'] == "Billing":
        return "handle_billing"
    else:
        return "handle_general"

In [ ]:
# Build LangGraph workflow
workflow = StateGraph(State)

workflow.add_node("categorize", categorize)
workflow.add_node("analyze_sentiment", analyze_sentiment)
workflow.add_node("handle_technical", handle_technical)
workflow.add_node("handle_billing", handle_billing)
workflow.add_node("handle_general", handle_general)
workflow.add_node("escalate", escalate)

workflow.set_entry_point("categorize")
workflow.add_edge("categorize", "analyze_sentiment")
workflow.add_conditional_edges(
    "analyze_sentiment",
    route_query,
    {
        "handle_technical": "handle_technical",
        "handle_billing": "handle_billing",
        "handle_general": "handle_general",
        "escalate": "escalate"
    }
)

workflow.add_edge("handle_technical", END)
workflow.add_edge("handle_billing", END)
workflow.add_edge("handle_general", END)
workflow.add_edge("escalate", END)

app = workflow.compile()

In [ ]:
# Visualize graph
display(Image(app.get_graph().draw_mermaid_png(draw_method=MermaidDrawMethod.API)))

In [ ]:
# Helper function to run agent
def run_customer_support(query: str) -> Dict[str, str]:
    result = app.invoke({"query": query})
    return {
        "Category": result['category'],
        "Sentiment": result['sentiment'],
        "Response": result['response']
    }

In [ ]:
# Gradio interface
import gradio as gr

def gradio_interface(query: str):
    result = run_customer_support(query)
    return (
        f"**Category:** {result['Category']}\n\n"
        f"**Sentiment:** {result['Sentiment']}\n\n"
        f"**Response:** {result['Response']}"
    )

iface = gr.Interface(
    fn=gradio_interface,
    inputs=gr.Textbox(lines=2, placeholder="Enter customer query..."),
    outputs=gr.Markdown(),
    title="Customer Support Agent",
    description="AI-powered customer support using LangGraph and Groq"
)

if __name__ == "__main__":
    iface.launch()